# Imports

In [2]:
import dijkstra
import astar
import pandas
import data_loader
from timer import Timer
import cli_utils

# Loading data

In [3]:
graph= data_loader.load_data_to_graph(normalize_coordinates=True)
graph_dict = graph.graph_dict

# Test data

In [4]:
bus_stops = [
    ('wrocławski park przemysłowy','pl. grunwaldzki'),
    ('grabiszyńska','pl. daniłowskiego'),
    ('kwiska','pl. grunwaldzki'),
    ('krzyki','leśnica'),
    ('leśnica','biskupin'),
    ('wrocławski park przemysłowy','dworzec autobusowy'),
    ('fat','osiedle sobieskiego')
]
bus_stops_test_data = [(graph_dict[start_stop], graph_dict[end_stop]) for start_stop, end_stop in bus_stops]

start_times = ['9:00','9:15','9:30', '2:00', '2:15', '2:30']
start_times_test_data = [cli_utils.normalize_input_time(time) for time in start_times]

# Data analysis

In [5]:
data = []
configs = {
    'Dijkstra for time' : lambda start_stop, end_stop, start_time: dijkstra.shortest_path(graph_dict=graph_dict, start_stop=start_stop.name.lower(), goal_stop=end_stop.name.lower(), start_time=start_time, cost_fn=dijkstra.DEFAULT_COST_FUNCTION),
    'A* for time' : lambda start_stop, end_stop, start_time: astar.astar(start=start_stop, goal=end_stop, start_time=start_time, cost_fn=astar.TIME_COST_FUNCTION, heuristic_fn=astar.TIME_HEURISTIC),
    'A* for line changes' : lambda start_stop, end_stop, start_time: astar.astar(start=start_stop, goal=end_stop, start_time=start_time, cost_fn=astar.LINE_CHANGE_COST_FUNCTION, heuristic_fn=astar.LINE_CHANGE_HEURISTIC)
}

timer = Timer()

for start_time in start_times_test_data:
    for start_stop, end_stop in bus_stops_test_data:
        for config_name, function in configs.items():
            
            cost, _ = timer.run(lambda : function(start_stop, end_stop, start_time))
            computation_time = timer.elapsed_time
            
            record = config_name, start_stop, end_stop, cli_utils.convert_normalized_time(start_time), cost, computation_time
            data.append(record)

columns = ['Algorithm', 'Start stop', 'End stop', 'Start time', 'Cost', 'Computation time']

statistics = pandas.DataFrame(data, columns=columns)
statistics

,Algorithm,Start stop,End stop,Start time,Cost,Computation time
0,Dijkstra for time,Wrocławski Park Przemysłowy,PL. GRUNWALDZKI,09:00,22.0,1.005848
1,A* for time,Wrocławski Park Przemysłowy,PL. GRUNWALDZKI,09:00,22.0,0.133310
2,A* for line changes,Wrocławski Park Przemysłowy,PL. GRUNWALDZKI,09:00,0.0,0.215631
3,Dijkstra for time,Grabiszyńska,pl. Daniłowskiego,09:00,29.0,1.044681
4,A* for time,Grabiszyńska,pl. Daniłowskiego,09:00,35.0,0.177930
...,...,...,...,...,...,...
121,A* for time,Wrocławski Park Przemysłowy,DWORZEC AUTOBUSOWY,02:30,20.0,0.060561
122,A* for line changes,Wrocławski Park Przemysłowy,DWORZEC AUTOBUSOWY,02:30,1.0,0.327316
123,Dijkstra for time,FAT,OSIEDLE SOBIESKIEGO,02:30,90.0,0.975145
124,A* for time,FAT,OSIEDLE SOBIESKIEGO,02:30,90.0,0.376247


In [6]:
time_optimization_statistics = statistics[statistics['Algorithm'] != 'A* for line changes']


pivoted_time_optimization_statistics = time_optimization_statistics.pivot_table(index=['Start stop', 'End stop', 'Start time'], columns='Algorithm', values=['Cost', 'Computation time'])
pivoted_time_optimization_statistics

Computation time  \
Algorithm                                                       A* for time   
Start stop                  End stop            Start time                    
FAT                         OSIEDLE SOBIESKIEGO 02:00              0.545037   
                                                02:15              0.430442   
                                                02:30              0.376247   
                                                09:00              0.266546   
                                                09:15              0.357450   
                                                09:30              0.272542   
Grabiszyńska                pl. Daniłowskiego   02:00              0.527520   
                                                02:15              0.375729   
                                                02:30              0.355928   
                                                09:00              0.177930   
                                                09:15              0.151772   
                                                09:30              0.157600   
KRZYKI                      LEŚNICA             02:00              0.720857   
                                                02:15              0.581717   
                                                02:30              0.599923   
                                                09:00              0.152508   
                                                09:15              0.278877   
                                                09:30              0.230407   
Kwiska                      PL. GRUNWALDZKI     02:00              0.327501   
                                                02:15              0.327492   
                                                02:30              0.332631   
                                                09:00              0.136596   
                                                09:15              0.145206   
                                                09:30              0.175691   
LEŚNICA                     BISKUPIN            02:00              0.539629   
                                                02:15              0.452127   
                                                02:30              0.457656   
                                                09:00              0.236480   
                                                09:15              0.260630   
                                                09:30              0.245837   
Wrocławski Park Przemysłowy DWORZEC AUTOBUSOWY  02:00              0.067097   
                                                02:15              0.077185   
                                                02:30              0.060561   
                                                09:00              0.131030   
                                                09:15              0.078763   
                                                09:30              0.102947   
                            PL. GRUNWALDZKI     02:00              0.254215   
                                                02:15              0.248470   
                                                02:30              0.270418   
                                                09:00              0.133310   
                                                09:15              0.084295   
                                                09:30              0.096934   

                                                                              \
Algorithm                                                  Dijkstra for time   
Start stop                  End stop            Start time                     
FAT                         OSIEDLE SOBIESKIEGO 02:00               0.964155   
                                                02:15               0.975501   
                                                02:30               0.975145   
                       

In [7]:
average_cost_per_algorithm_time_optimization = pivoted_time_optimization_statistics['Cost'].mean()

# Calculate average for computation time per algorithm
average_computation_time_per_algorithm_time_optimization = pivoted_time_optimization_statistics['Computation time'].mean()

# Create a new DataFrame to store the averages
average_statistics_time_optimization = pandas.DataFrame({
    'Average Cost': average_cost_per_algorithm_time_optimization,
    'Average Computation Time': average_computation_time_per_algorithm_time_optimization
})

average_statistics_time_optimization

,Average Cost,Average Computation Time
Algorithm,,
A* for time,56.261905,0.280994
Dijkstra for time,51.285714,1.000190


In [11]:
line_changes_optimization_statistics = statistics[statistics['Algorithm'] == 'A* for line changes']

# Update 'Cost' if greater than or equal to 1000 and remove records where 'Cost' is infinity
line_changes_optimization_statistics = line_changes_optimization_statistics[line_changes_optimization_statistics['Cost'] != float('inf')]
line_changes_optimization_statistics.loc[line_changes_optimization_statistics['Cost'] >= 1000, 'Cost'] -= 1000

line_changes_optimization_statistics

,Algorithm,Start stop,End stop,Start time,Cost,Computation time
2,A* for line changes,Wrocławski Park Przemysłowy,PL. GRUNWALDZKI,09:00,0.0,0.215631
5,A* for line changes,Grabiszyńska,pl. Daniłowskiego,09:00,1.0,0.185928
8,A* for line changes,Kwiska,PL. GRUNWALDZKI,09:00,0.0,0.345246
11,A* for line changes,KRZYKI,LEŚNICA,09:00,3.0,0.586873
14,A* for line changes,LEŚNICA,BISKUPIN,09:00,0.0,0.308913
17,A* for line changes,Wrocławski Park Przemysłowy,DWORZEC AUTOBUSOWY,09:00,1.0,0.789959
20,A* for line changes,FAT,OSIEDLE SOBIESKIEGO,09:00,1.0,0.242459
23,A* for line changes,Wrocławski Park Przemysłowy,PL. GRUNWALDZKI,09:15,0.0,0.206276
26,A* for line changes,Grabiszyńska,pl. Daniłowskiego,09:15,1.0,0.193885
29,A* for line changes,Kwiska,PL. GRUNWALDZKI,09:15,0.0,0.321147


In [15]:
average_cost_per_algorithm_line_changes = line_changes_optimization_statistics['Cost'].mean()

# Calculate average for computation time per algorithm
average_computation_time_per_algorithm_line_changes = line_changes_optimization_statistics['Computation time'].mean()

# Create a new DataFrame to store the averages
average_statistics_line_changes = pandas.DataFrame({
    'Average Cost': [average_cost_per_algorithm_line_changes],
    'Average Computation Time': [average_computation_time_per_algorithm_line_changes]
})

average_statistics_line_changes

,Average Cost,Average Computation Time
0,1.564103,0.448149
